In [1]:
"""
Feature: urine_output_ml
Extract total urine output in ml from outputevents
"""
import pandas as pd
import os
from config_paths import *
from utils import drop_columns

In [2]:
# Read dataset
df = pd.read_csv(dataset_path)
initial_row_count = len(df)
df["admittime"] = pd.to_datetime(df["admittime"])
df["dischtime"] = pd.to_datetime(df["dischtime"])

# Load outputevents
outputevents = pd.read_csv(
    os.path.join(icu_path, "outputevents.csv"),
    usecols=["subject_id", "hadm_id", "itemid", "charttime", "value"]
)
outputevents["charttime"] = pd.to_datetime(outputevents["charttime"])

# Urine output item IDs
urine_output_itemids = [226627, 226631, 227519]

# Filter and merge
urine_output = outputevents[outputevents["itemid"].isin(urine_output_itemids)].copy()
urine_output = urine_output.merge(
    df[["subject_id", "hadm_id", "admittime", "dischtime"]],
    on=["subject_id", "hadm_id"],
    how="left"
)

# Restrict to admission window
urine_output = urine_output[
    (urine_output["charttime"] >= urine_output["admittime"]) &
    (urine_output["charttime"] <= urine_output["dischtime"])
]

# Convert to numeric
urine_output["value"] = pd.to_numeric(urine_output["value"], errors="coerce")

# Sum per admission
urine_output_sum = (
    urine_output.groupby(["subject_id", "hadm_id"])["value"]
    .sum()
    .reset_index()
    .rename(columns={"value": "urine_output_ml"})
)

# Merge
df = df.merge(urine_output_sum, on=["subject_id", "hadm_id"], how="left")

# Save
df.to_csv(dataset_path, index=False)

# Diagnostic output
print(f"\nTotal rows: {len(df)}")
print(f"Expected total: {initial_row_count}")
if len(df) == initial_row_count:
    print(f"✓ Row count matches expected total ({initial_row_count})")
else:
    print(f"⚠ Row count mismatch! Expected {initial_row_count}, got {len(df)} (difference: {len(df) - initial_row_count})")
print(f"Dataset shape: {df.shape}")
print(f"Feature 'urine_output_ml' added. Non-null count: {df['urine_output_ml'].notna().sum()}")
print(f"Dataset shape: {df.shape}")




Total rows: 158020
Expected total: 158020
✓ Row count matches expected total (158020)
Dataset shape: (158020, 85)
Feature 'urine_output_ml' added. Non-null count: 20968
Dataset shape: (158020, 85)


In [3]:
df.columns

Index(['subject_id', 'hadm_id', 'cauti_type', 'cauti_icd_codes',
       'other_uti_icd_codes', 'remaining_icd_codes', 'gender', 'anchor_age',
       'admittime', 'dischtime', 'admission_type', 'admission_location',
       'discharge_location', 'race', 'catheter_procedures',
       'catheter_procedures_ir', 'catheter_used_in_procedures_ir',
       'catheter_insertion_date_ir', 'catheter_removal_date_ir',
       'catheter_outputevents', 'catheter_used_in_output_events',
       'datetimeevents', 'catheter_used_in_datetime_events',
       'insertion_date_from_datetimeevents',
       'removal_date_from_datetimeevents', 'catheter_procedure_events',
       'catheter_used_in_procedure_events', 'catheter_size_from_chartevents',
       'index', 'catheter_clinical_notes', 'catheter_used_in_clinical_notes',
       'catheter_present', 'final_cauti_flag', 'final_insertion_date',
       'final_removal_date', 'catheter_duration_days', 'BMI_in_admission',
       'BMI_computed', 'BMI_last_year', 'BMI', 

In [4]:
# cols_to_drop = [
#  "urine_output_ml"
# ]
# df = drop_columns(df, cols_to_drop)
# df.to_csv(dataset_path, index=False)